In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

# = = = = = = = = = = = = = = =

import pandas as pd

output_correct = pd.DataFrame()
output_error = pd.DataFrame()

# = = = = = = = = = = = = = = =

input_ = pd.read_excel('./part_number.xlsx',
                       header=0,
                       dtype=str).fillna('')

print('总数量：' + str(len(input_)))
print()

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
import UA
import Proxy

# = = = = = = = = = = = = = = =

import json

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for a in range(len(input_)):
    work.put_nowait(a)
    
# = = = = = = = = = = = = = = =

def crawler():
    global output_correct, output_error
    
    while not work.empty():
        a = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        crawler_status = 'error'
        for b in range(100):
            try:
                url = 'https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?part=' + input_.loc[a, 'basePart'] + '&func=PART&vid='

                resp = requests.get(url,
                                    headers=UA.get_User_Agent_Requests(),
                                    proxies=Proxy.get_Proxy_Requests()).text
                
                # = = = = = = = = = = = = = = =
                
                json_data = json.loads(resp)
                
                # = = = = = = = = = = = = = = =
                
                vehicle = '\n'.join([guide['buyersGuideDesc'].strip() for guide in json_data['pp']['buyersGuides']])

                # = = = = = = = = = = = = = = =
                
                df_temp = pd.DataFrame([{'序号': int(input_.loc[a, 'rowId']),
                                         'Part_Number': json_data['pd']['basePart'].strip(),
                                         'Category': '' if len(json_data['pp']['categories'])==0 else json_data['pp']['categories'][0]['categoryName_en'].strip(),
                                         'Part_Type_Short': json_data['pd']['partDesc_en'].strip(),
                                         'Part_Type_Long': json_data['pd']['partLongDesc_en'].strip(),
                                         'Brand': json_data['pd']['brand'].strip(),
                                         'POP': json_data['pd']['pop'].strip(),
                                         'Per_Car': json_data['pd']['qtyEach'],
                                         'Vehicle': vehicle,
                                         'Pic': '',
                                         'Url': url,
                                         'Src': '',
                                         'JOIN_PNB': json_data['pd']['basePart'].strip() + ';' + json_data['pd']['brand'].strip()}])

                for specification in json_data['pp']['partSpecs']:
                    df_temp.loc[0, specification['attributeName_en'].lower().strip()] = specification['siteValue_en']

                output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

                # = = = = = = = = = = = = = = =
                
                crawler_status = 'ok'
                
                break
            except:
                time.sleep(0.3)
                continue
        
        # = = = = = = = = = = = = = = =
        
        if crawler_status == 'error':
            df_temp = pd.DataFrame([{'rowId': int(input_.loc[a, 'rowId']),
                                     'basePart': input_.loc[a, 'basePart']}])

            output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')
            
        # = = = = = = = = = = = = = = =
        
        print('[' + crawler_status + '] - ' + input_.loc[a, 'basePart'] +  '\n[尝试次数：' + str(b+1) + '] - [剩余数量：' + str(work.qsize()) + '] - [当前时间：' + datetime.now().strftime('%H:%M:%S') + ']\n')
        
# = = = = = = = = = = = = = = =

list_task = []
for _ in range(20):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print('输出ing...')
print()
output_correct = output_correct.drop_duplicates(ignore_index=True)
output_correct = output_correct.sort_values(by=['序号'], ascending=[True]).reset_index(drop=True)
output_correct.to_excel('./part-' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
if len(output_error) != 0:
    output_error = output_error.drop_duplicates(ignore_index=True)
    output_error = output_error.sort_values(by=['basePart'], ascending=[True]).reset_index(drop=True)
    output_error.to_excel('./part_error.xlsx', index=False)
    print('爬虫存在error')
    print()
print('Done ~')

总数量：1141

[ok] - ATS35
[尝试次数：1] - [剩余数量：1121] - [当前时间：13:54:16]

[ok] - AS415
[尝试次数：1] - [剩余数量：1120] - [当前时间：13:54:16]

[ok] - ATS32
[尝试次数：1] - [剩余数量：1119] - [当前时间：13:54:16]

[ok] - AS445
[尝试次数：1] - [剩余数量：1118] - [当前时间：13:54:16]

[ok] - AS432
[尝试次数：1] - [剩余数量：1117] - [当前时间：13:54:16]

[ok] - AS493
[尝试次数：1] - [剩余数量：1116] - [当前时间：13:54:16]

[ok] - AX1
[尝试次数：1] - [剩余数量：1115] - [当前时间：13:54:16]

[ok] - ATS50
[尝试次数：1] - [剩余数量：1114] - [当前时间：13:54:16]

[ok] - AS420
[尝试次数：1] - [剩余数量：1113] - [当前时间：13:54:16]

[ok] - ATS24
[尝试次数：1] - [剩余数量：1112] - [当前时间：13:54:16]

[ok] - ATS52
[尝试次数：1] - [剩余数量：1111] - [当前时间：13:54:17]

[ok] - ATS44
[尝试次数：1] - [剩余数量：1110] - [当前时间：13:54:17]

[ok] - ATS45
[尝试次数：1] - [剩余数量：1109] - [当前时间：13:54:17]

[ok] - AX3T
[尝试次数：1] - [剩余数量：1108] - [当前时间：13:54:17]

[ok] - AX2
[尝试次数：1] - [剩余数量：1107] - [当前时间：13:54:17]

[ok] - AS185
[尝试次数：1] - [剩余数量：1106] - [当前时间：13:54:17]

[ok] - AX4
[尝试次数：1] - [剩余数量：1105] - [当前时间：13:54:17]

[ok] - AX6
[尝试次数：1] - [剩余数量：1104] - [当前时间：13:54:17]

[ok] - AX

[ok] - AX185
[尝试次数：5] - [剩余数量：970] - [当前时间：13:54:37]

[ok] - AX186
[尝试次数：3] - [剩余数量：969] - [当前时间：13:54:37]

[ok] - AX191
[尝试次数：3] - [剩余数量：968] - [当前时间：13:54:37]

[ok] - AX199
[尝试次数：1] - [剩余数量：967] - [当前时间：13:54:38]

[ok] - AX181
[尝试次数：9] - [剩余数量：966] - [当前时间：13:54:38]

[ok] - AX190
[尝试次数：7] - [剩余数量：965] - [当前时间：13:54:38]

[ok] - AX171
[尝试次数：10] - [剩余数量：964] - [当前时间：13:54:39]

[ok] - AX134
[尝试次数：29] - [剩余数量：963] - [当前时间：13:54:39]

[ok] - AX194
[尝试次数：4] - [剩余数量：962] - [当前时间：13:54:39]

[ok] - AX-198
[尝试次数：3] - [剩余数量：961] - [当前时间：13:54:39]

[ok] - AX202
[尝试次数：4] - [剩余数量：960] - [当前时间：13:54:39]

[ok] - AX197
[尝试次数：7] - [剩余数量：959] - [当前时间：13:54:40]

[ok] - AX208
[尝试次数：3] - [剩余数量：958] - [当前时间：13:54:40]

[ok] - AX214
[尝试次数：2] - [剩余数量：957] - [当前时间：13:54:40]

[ok] - AX212
[尝试次数：3] - [剩余数量：956] - [当前时间：13:54:40]

[ok] - AX207
[尝试次数：5] - [剩余数量：955] - [当前时间：13:54:41]

[ok] - AX-196
[尝试次数：1] - [剩余数量：954] - [当前时间：13:54:41]

[ok] - AX200
[尝试次数：3] - [剩余数量：953] - [当前时间：13:54:41]

[ok] - AX205
[尝试次数：4] - 

[ok] - AX380
[尝试次数：1] - [剩余数量：818] - [当前时间：13:55:10]

[ok] - AX377
[尝试次数：1] - [剩余数量：817] - [当前时间：13:55:10]

[ok] - AX385
[尝试次数：1] - [剩余数量：816] - [当前时间：13:55:10]

[ok] - AX387
[尝试次数：1] - [剩余数量：815] - [当前时间：13:55:11]

[ok] - AX379
[尝试次数：1] - [剩余数量：814] - [当前时间：13:55:11]

[ok] - AX386
[尝试次数：1] - [剩余数量：813] - [当前时间：13:55:11]

[ok] - AX392
[尝试次数：1] - [剩余数量：812] - [当前时间：13:55:12]

[ok] - AX393
[尝试次数：1] - [剩余数量：811] - [当前时间：13:55:12]

[ok] - AX354
[尝试次数：1] - [剩余数量：810] - [当前时间：13:55:13]

[ok] - AX395
[尝试次数：1] - [剩余数量：809] - [当前时间：13:55:13]

[ok] - AX373
[尝试次数：1] - [剩余数量：808] - [当前时间：13:55:13]

[ok] - AX333
[尝试次数：2] - [剩余数量：807] - [当前时间：13:55:14]

[ok] - AX398
[尝试次数：1] - [剩余数量：806] - [当前时间：13:55:15]

[ok] - AX397
[尝试次数：1] - [剩余数量：805] - [当前时间：13:55:15]

[ok] - AX400
[尝试次数：1] - [剩余数量：804] - [当前时间：13:55:15]

[ok] - AX383
[尝试次数：1] - [剩余数量：803] - [当前时间：13:55:15]

[ok] - AX390
[尝试次数：1] - [剩余数量：802] - [当前时间：13:55:16]

[ok] - AX402
[尝试次数：1] - [剩余数量：801] - [当前时间：13:55:16]

[ok] - AX404
[尝试次数：1] - [剩余数

[ok] - ETS7
[尝试次数：1] - [剩余数量：666] - [当前时间：13:55:46]

[ok] - ETS34
[尝试次数：1] - [剩余数量：665] - [当前时间：13:55:46]

[ok] - ETS22
[尝试次数：1] - [剩余数量：664] - [当前时间：13:55:46]

[ok] - ETS37
[尝试次数：1] - [剩余数量：663] - [当前时间：13:55:47]

[ok] - ETS35
[尝试次数：1] - [剩余数量：662] - [当前时间：13:55:47]

[ok] - ETS5
[尝试次数：1] - [剩余数量：661] - [当前时间：13:55:47]

[ok] - AX414
[尝试次数：1] - [剩余数量：660] - [当前时间：13:55:48]

[ok] - ETS12
[尝试次数：1] - [剩余数量：659] - [当前时间：13:55:48]

[ok] - AX652
[尝试次数：1] - [剩余数量：658] - [当前时间：13:55:48]

[ok] - ETS46
[尝试次数：1] - [剩余数量：657] - [当前时间：13:55:48]

[ok] - ETS47
[尝试次数：1] - [剩余数量：656] - [当前时间：13:55:48]

[ok] - ETS44
[尝试次数：1] - [剩余数量：655] - [当前时间：13:55:49]

[ok] - ETS38
[尝试次数：1] - [剩余数量：654] - [当前时间：13:55:49]

[ok] - ETS49
[尝试次数：1] - [剩余数量：653] - [当前时间：13:55:49]

[ok] - ETS45
[尝试次数：1] - [剩余数量：652] - [当前时间：13:55:49]

[ok] - ETS50
[尝试次数：1] - [剩余数量：651] - [当前时间：13:55:49]

[ok] - ETS52
[尝试次数：1] - [剩余数量：650] - [当前时间：13:55:49]

[ok] - ETS43
[尝试次数：1] - [剩余数量：649] - [当前时间：13:55:50]

[ok] - ETS59
[尝试次数：1] - [剩余数量：

[ok] - ETS188
[尝试次数：1] - [剩余数量：514] - [当前时间：13:56:15]

[ok] - ETS211
[尝试次数：1] - [剩余数量：513] - [当前时间：13:56:15]

[ok] - ETS217
[尝试次数：1] - [剩余数量：512] - [当前时间：13:56:16]

[ok] - ETS25
[尝试次数：1] - [剩余数量：511] - [当前时间：13:56:16]

[ok] - ETS219
[尝试次数：1] - [剩余数量：510] - [当前时间：13:56:16]

[ok] - ETS205
[尝试次数：1] - [剩余数量：509] - [当前时间：13:56:16]

[ok] - ETS220
[尝试次数：1] - [剩余数量：508] - [当前时间：13:56:16]

[ok] - ETS218
[尝试次数：1] - [剩余数量：507] - [当前时间：13:56:16]

[ok] - ETS216
[尝试次数：1] - [剩余数量：506] - [当前时间：13:56:16]

[ok] - ETS200
[尝试次数：1] - [剩余数量：505] - [当前时间：13:56:16]

[ok] - ETS196
[尝试次数：1] - [剩余数量：504] - [当前时间：13:56:16]

[ok] - ETS221
[尝试次数：1] - [剩余数量：503] - [当前时间：13:56:16]

[ok] - ETS198
[尝试次数：1] - [剩余数量：502] - [当前时间：13:56:16]

[ok] - ETS224
[尝试次数：1] - [剩余数量：501] - [当前时间：13:56:17]

[ok] - ETS226
[尝试次数：1] - [剩余数量：500] - [当前时间：13:56:17]

[ok] - ETS206
[尝试次数：1] - [剩余数量：499] - [当前时间：13:56:17]

[ok] - ETS229
[尝试次数：1] - [剩余数量：498] - [当前时间：13:56:17]

[ok] - ETS227
[尝试次数：1] - [剩余数量：497] - [当前时间：13:56:17]

[ok] - ETS2

[ok] - TS-24
[尝试次数：1] - [剩余数量：364] - [当前时间：13:56:46]

[ok] - S2945
[尝试次数：1] - [剩余数量：363] - [当前时间：13:56:47]

[ok] - TS-73
[尝试次数：1] - [剩余数量：362] - [当前时间：13:56:47]

[ok] - TS73T
[尝试次数：1] - [剩余数量：361] - [当前时间：13:56:47]

[ok] - TS71T
[尝试次数：1] - [剩余数量：360] - [当前时间：13:56:47]

[ok] - AX484
[尝试次数：2] - [剩余数量：359] - [当前时间：13:56:48]

[ok] - TS76T
[尝试次数：1] - [剩余数量：358] - [当前时间：13:56:48]

[ok] - TS-76
[尝试次数：1] - [剩余数量：357] - [当前时间：13:56:48]

[ok] - TS-174
[尝试次数：1] - [剩余数量：356] - [当前时间：13:56:48]

[ok] - HP3895
[尝试次数：1] - [剩余数量：355] - [当前时间：13:56:49]

[ok] - TS-83
[尝试次数：1] - [剩余数量：354] - [当前时间：13:56:49]

[ok] - TS-233
[尝试次数：1] - [剩余数量：353] - [当前时间：13:56:49]

[ok] - TS-181
[尝试次数：1] - [剩余数量：352] - [当前时间：13:56:49]

[ok] - TS-205
[尝试次数：1] - [剩余数量：351] - [当前时间：13:56:49]

[ok] - TS-235
[尝试次数：1] - [剩余数量：350] - [当前时间：13:56:50]

[ok] - TS-263
[尝试次数：1] - [剩余数量：349] - [当前时间：13:56:50]

[ok] - TS-249
[尝试次数：1] - [剩余数量：348] - [当前时间：13:56:50]

[ok] - TS-253
[尝试次数：1] - [剩余数量：347] - [当前时间：13:56:50]

[ok] - TS253T
[尝试次数

[ok] - TX89
[尝试次数：1] - [剩余数量：210] - [当前时间：13:57:24]

[ok] - TX87
[尝试次数：1] - [剩余数量：209] - [当前时间：13:57:24]

[ok] - TX95
[尝试次数：1] - [剩余数量：208] - [当前时间：13:57:24]

[ok] - TX97
[尝试次数：1] - [剩余数量：207] - [当前时间：13:57:24]

[ok] - TX96
[尝试次数：1] - [剩余数量：206] - [当前时间：13:57:25]

[ok] - TX98
[尝试次数：1] - [剩余数量：205] - [当前时间：13:57:25]

[ok] - TX81T
[尝试次数：1] - [剩余数量：204] - [当前时间：13:57:25]

[ok] - TX102
[尝试次数：1] - [剩余数量：203] - [当前时间：13:57:25]

[ok] - TX98T
[尝试次数：1] - [剩余数量：202] - [当前时间：13:57:25]

[ok] - TX78
[尝试次数：1] - [剩余数量：201] - [当前时间：13:57:26]

[ok] - TS1000
[尝试次数：2] - [剩余数量：200] - [当前时间：13:57:26]

[ok] - TX104T
[尝试次数：1] - [剩余数量：199] - [当前时间：13:57:26]

[ok] - TX106
[尝试次数：1] - [剩余数量：198] - [当前时间：13:57:26]

[ok] - TX107
[尝试次数：1] - [剩余数量：197] - [当前时间：13:57:26]

[ok] - TX104
[尝试次数：1] - [剩余数量：196] - [当前时间：13:57:26]

[ok] - TX106T
[尝试次数：1] - [剩余数量：195] - [当前时间：13:57:26]

[ok] - TX108T
[尝试次数：1] - [剩余数量：194] - [当前时间：13:57:27]

[ok] - TX109
[尝试次数：1] - [剩余数量：193] - [当前时间：13:57:27]

[ok] - ETS162
[尝试次数：2] - [剩余数量：

[ok] - S2495
[尝试次数：2] - [剩余数量：57] - [当前时间：13:57:58]

[ok] - TX278
[尝试次数：1] - [剩余数量：56] - [当前时间：13:57:58]

[ok] - TX300
[尝试次数：1] - [剩余数量：55] - [当前时间：13:57:58]

[ok] - TX247
[尝试次数：1] - [剩余数量：54] - [当前时间：13:57:58]

[ok] - TX303
[尝试次数：1] - [剩余数量：53] - [当前时间：13:57:58]

[ok] - TX305
[尝试次数：1] - [剩余数量：52] - [当前时间：13:57:58]

[ok] - TX307
[尝试次数：1] - [剩余数量：51] - [当前时间：13:57:59]

[ok] - TX304
[尝试次数：1] - [剩余数量：50] - [当前时间：13:57:59]

[ok] - TX316
[尝试次数：1] - [剩余数量：49] - [当前时间：13:57:59]

[ok] - TX310
[尝试次数：1] - [剩余数量：48] - [当前时间：13:58:00]

[ok] - TX282
[尝试次数：1] - [剩余数量：47] - [当前时间：13:58:00]

[ok] - TX318
[尝试次数：1] - [剩余数量：46] - [当前时间：13:58:00]

[ok] - TX288
[尝试次数：1] - [剩余数量：45] - [当前时间：13:58:00]

[ok] - TX311
[尝试次数：1] - [剩余数量：44] - [当前时间：13:58:00]

[ok] - TX321
[尝试次数：1] - [剩余数量：43] - [当前时间：13:58:00]

[ok] - TX322
[尝试次数：1] - [剩余数量：42] - [当前时间：13:58:00]

[ok] - TX319
[尝试次数：1] - [剩余数量：41] - [当前时间：13:58:01]

[ok] - TX323
[尝试次数：1] - [剩余数量：40] - [当前时间：13:58:01]

[ok] - TX326
[尝试次数：1] - [剩余数量：39] - [当前时间：13:5